In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import os
from time import time

from model_toy import get_toy_ResNet

root_logs = os.path.join('logs', 'custom')

# Get the data

In [2]:
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train_full, x_test = x_train_full/255., x_test/255.

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_val,  y_train, y_val  = train_test_split(x_train_full, y_train_full)

## Define the toy ResNet

In [4]:
model = get_toy_ResNet()
model.build(x_train.shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  896       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  9216      
_________________________________________________________________
batch_normalization (BatchNo multiple                  128       
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
res_block (Res_block)        multiple                  18688     
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  1

# Training

In [5]:
epochs     = 10
batch_size = 32
loss_fn    = tf.keras.losses.sparse_categorical_crossentropy
optimizer  = tf.keras.optimizers.RMSprop()
acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

## Keras APIs

In [6]:
model = get_toy_ResNet()

model.compile(loss=loss_fn, optimizer=optimizer, metrics=[acc_metric])
start = time()
history = model.fit(x=x_train, y=y_train,
                    batch_size=batch_size,
                    epochs=epochs, verbose=1,
                    validation_data=(x_val, y_val))
stop = time()
print("%d epochs in %.2fs"%(epochs, stop-start))

Epoch 1/10
1172/1172 [==============================] - 8s 7ms/step - loss: 1.9144 - sparse_categorical_accuracy: 0.2926 - val_loss: 1.7986 - val_sparse_categorical_accuracy: 0.3687
Epoch 2/10
1172/1172 [==============================] - 7s 6ms/step - loss: 1.4954 - sparse_categorical_accuracy: 0.4707 - val_loss: 1.5852 - val_sparse_categorical_accuracy: 0.4982
Epoch 3/10
1172/1172 [==============================] - 7s 6ms/step - loss: 1.3353 - sparse_categorical_accuracy: 0.5423 - val_loss: 1.3281 - val_sparse_categorical_accuracy: 0.5313
Epoch 4/10
1172/1172 [==============================] - 7s 6ms/step - loss: 1.2062 - sparse_categorical_accuracy: 0.5871 - val_loss: 1.3513 - val_sparse_categorical_accuracy: 0.5542
Epoch 5/10
1172/1172 [==============================] - 7s 6ms/step - loss: 1.1239 - sparse_categorical_accuracy: 0.6220 - val_loss: 1.0842 - val_sparse_categorical_accuracy: 0.6418
Epoch 6/10
1172/1172 [==============================] - 7s 6ms/step - loss: 1.0638 - spars

## Custom training

In [7]:
def get_batch(batch_size):
    idx = np.random.randint(low=0, high=len(x_train), size=batch_size)
    return x_train[idx], y_train[idx]

@tf.function
def train_step(model, loss_fn, optimizer, x_batch, y_batch):
    with tf.GradientTape() as tape:
        # Forward
        y_pred   = model(x_batch, training=True)
        out_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        tot_loss = tf.add_n([out_loss] + model.losses)
    # Backward    
    gradients = tape.gradient(tot_loss, model.trainable_variables)
    # Update
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return y_pred, tot_loss

def predict(model, loss_fn, x_batch, y_batch):
    y_pred   = model(x_batch, training=False)
    out_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
    tot_loss = tf.add_n([out_loss] + model.losses)
    return y_pred, tot_loss

In [8]:
model = get_toy_ResNet()
# Metrics
acc_metric.reset_states()
loss_metric = tf.keras.metrics.Mean()

start_t = time()
for epoch in range(epochs):
    start = time()
    # Train
    for i in range(0,len(x_train),batch_size):
        x_batch, y_batch = get_batch(batch_size)
        y_pred, loss     = train_step(model, loss_fn, optimizer, x_batch, y_batch)
        acc_metric.update_state(y_batch, y_pred)    
    train_acc = acc_metric.result()
    acc_metric.reset_states()
    stop = time()
    
    # Validate
    for i in range(0,len(x_val),batch_size):
        x_batch, y_batch = x_val[i:i+batch_size], y_val[i:i+batch_size]
        y_pred, loss     = predict(model, loss_fn, x_batch, y_batch)
        acc_metric.update_state(y_batch, y_pred)
        loss_metric.update_state(loss)
    val_acc  = acc_metric.result()
    loss_acc = loss_metric.result()
    acc_metric.reset_states()    
    loss_metric.reset_states()    
    print("Epoch %d/%d: %.4fs\tTrain: accuracy: %.3f - last loss: %.3f\tValidation: accuracy %.3f - mean loss %.3f"%
          (epoch, epochs, stop - start, train_acc, loss, val_acc, loss_acc))
stop_t = time()    
print("%d epochs in %.2fs"%(epochs, stop_t-start_t))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0/10: 8.7333s	Train: accuracy: 0.314 - last loss: 1.415	Validation: accuracy 0.458 - mean loss 1.568
Epoch 1/10: 7.1448s	Train: accuracy: 0.482 - last loss: 1.008	Validation: accuracy 0.570 - mean loss 1.228
Epoch 2/10: 7.1412s	Train: accuracy: 0.556 - last loss: 1.869	Validation: accuracy 0.562 - mean loss 1.374
Epoch 3/10: 7.1372s	Train: accuracy: 0.601 - last loss: 0.860	Validation: accuracy 0.655 - mean loss 1.053
Epoch 4/10: 7.1305s	Train: accuracy: 0.635 - last loss: 1.008	Validation: accura